In [1]:
import pandas as pd
import os
from pathlib import Path
from sec_edgar_downloader import Downloader #https://pypi.org/project/sec-edgar-downloader/

## Define useful functions

In [2]:
def checkCreateFolder(fundName):
    #check if fund folder exists in /Data/, if not create one
    #also check if csv file exists in /Data/[fund_name]/, if not create it
    formattedFundName = fundName.replace(' ', '_')
        #create fund folder if it does not exist
    fundPath = os.path.abspath(os.path.join('..','Data', formattedFundName))
    Path(fundPath).mkdir(parents=True, exist_ok=True)
    #create csv with fund positions if does not exist
    fileName = formattedFundName + '.csv'
    if fileName not in os.listdir(fundPath):
        filePath = os.path.abspath(os.path.join('..','Data', formattedFundName, fileName))
        with(open(filePath, 'w')) as empty_csv:
            pass #will create the file if it does not exist

In [3]:
def lastAvailableData(fundName):
    #for a given fund name will check when is the last available report
    #will return the date of the last available report
    formattedFundName = fundName.replace(' ', '_')
    filePath = os.path.abspath(os.path.join('..','Data', formattedFundName, formattedFundName + '.csv'))
    try:
        data = pd.read_csv(filePath)
        #TODO download only the required reports if there are some reports available
        #process shit
        #return shit
    except pd.errors.EmptyDataError:
        return(None)

In [4]:
def downloadFiles(startDate, sec_id, fundName):
    #will download all required 13F-HR reports for a given sec_id
    #if None is passed as startDate, will download everything available
    #if a date is passed as startDate in format yyyymmdd it will download from that date
    formattedFundName = fundName.replace(' ', '_')
    downloadPath = os.path.abspath(os.path.join('..','Data', formattedFundName))
    #inialize downloader instance
    dl = Downloader(downloadPath)
    #if no date was passed, download all reports
    if startDate == None:
        dl.get("13F-HR",sec_id)
    else:
        dl.get("13F-HR",sec_id, after_date = startDate)
    
    
    

## Get all funds to download

In [5]:
fundlistPath = os.path.abspath(os.path.join('..', 'Funds.xlsx')) #read funds.xlsx spreadsheet at root folder
fundsDf = pd.read_excel (fundlistPath)

## Loop over each fund

In [7]:
for i in range(len(fundsDf.index)):
    #get fund name and ID
    fundName = fundsDf['FUND'].iloc[i]
    fundId = fundsDf['SEC ID'].iloc[i]
    checkCreateFolder(fundName) #call checkCreateFolder to create folders and files that are missing
    startDate = lastAvailableData(fundName) #get last available data, if there is no csv file create it
    downloadFiles(startDate = startDate, sec_id = fundId,fundName = fundName)
    
    